In [ ]:
%matplotlib inline                                                         
%run /home/fenics/fenics-matplotlib.py                                                                                                                                                                                                                        
from dolfin import *; from mshr import *; from IPython.display import display, clear_output; import time
import logging; logging.getLogger('FFC').setLevel(logging.WARNING)

import numpy as np

set_log_active(False)

def plot_compact(u, t, stepcounter): # Compact plot utility function
    if stepcounter % 5 == 0:
        ax.cla(); fig = plt.gcf(); fig.set_size_inches(16, 6.5)
        plt.subplot(1, 2, 1); mplot_function(u); plt.title("Wave") # Plot norm of velocity
        plt.colorbar(); # plt.axis(G)
        plt.subplot(1, 2, 2);
        plt.triplot(mesh2triang(mesh)); plt.title("Mesh") # Plot mesh
        plt.suptitle("Wave - t: %f" % (t)); plt.tight_layout(); clear_output(wait=True); display(pl)   
XMIN, XMAX = -1.0, 1.0;
YMIN, YMAX = -1.0, 1.0;
G = [XMIN, XMAX, YMIN, YMAX];
mresolution = 20;

mesh = RectangleMesh(Point(XMIN,YMIN), Point(XMAX, YMAX), mresolution, mresolution, 'crossed')

Q = FunctionSpace(mesh, "CG", 1); W = Q * Q; h = CellSize(mesh);
(p, q) = TestFunctions(W); w = Function(W); (u, v) = (w[0], w[1]);
u0 = Function(Q); v0 = Function(Q);

k = 0.000005; t = 0; T = 0.001
a = 3000;

eps = 1e-2

source_u = Expression("sin(-a * 10* t)", a=a, t=0, domain=mesh);
source_v = Expression("-a*10*cos(-a * 10* t)", a=a, t=0, domain=mesh);

mk0 = Expression("(sqrt((x[0]-xc)*(x[0]-xc)+(x[1]-yc)*(x[1]-yc))<eps)", xc=(XMIN+XMAX)/2.0 , yc=(YMIN+YMAX)/2.0, eps=eps, domain=mesh);
mk1 = Expression("(sqrt((x[0]-xc)*(x[0]-xc)+(x[1]-yc)*(x[1]-yc))<eps)", xc=XMIN , yc=(YMIN+YMAX)/2.0, eps=eps, domain=mesh);
mk2 = Expression("(sqrt((x[0]-xc)*(x[0]-xc)+(x[1]-yc)*(x[1]-yc))<eps)", xc=XMAX , yc=(YMIN+YMAX)/2.0, eps=eps, domain=mesh);
mk3 = Expression("(sqrt((x[0]-xc)*(x[0]-xc)+(x[1]-yc)*(x[1]-yc))<eps)", xc=(XMIN+XMAX)/2.0 , yc=YMIN, eps=eps, domain=mesh);
mk4 = Expression("(sqrt((x[0]-xc)*(x[0]-xc)+(x[1]-yc)*(x[1]-yc))<eps)", xc=(XMIN+XMAX)/2.0 , yc=YMAX, eps=eps, domain=mesh);

mk = mk1

gamma = 1e6
theta = 0.5;
while t < T: # Time-stepping loop        
    vm = theta*v + (1.0-theta)*v0;
    um = theta*u + (1.0-theta)*u0;
    source_u.t = t
    source_v.t = t
    
    ru_bc = gamma*mk*(u-source_u)*p*dx
    rv_bc = gamma*mk*(v-source_v)*q*dx
    
    ru=(u-u0)/k*p*dx - vm*p*dx + ru_bc
    rv=(v-v0)/k*q*dx + a*a*inner(grad(um),grad(q))*dx + rv_bc
    r = ru + rv;
    
    solve(r==0, w);
    
    # Shift to next timestep                                                     
    u0 = project(u, Q);
    v0 = project(v, Q);

    # visualize the solution
    pl, ax = plt.subplots();
    plot_compact(u0, t, 0)
    plt.close();
    t += k;